In [76]:
import torch 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [77]:
torch.cuda.get_device_name(id)

'NVIDIA GeForce MX150'

In [78]:
new_label_map = {
    '2': '0',
    '6': '1',
    '7': '2',
    '9': '3',
    '11': '4',
    '12': '5',
    '14': '6',
    '16': '7',
}

# clean dataset and create new label files and test,train .csv

In [79]:
import os
import csv
import string

label_directory = '/mnt/c/Projects/animal_detection/PASCAL_VOC/labels'

delete_ints = ['0','1','3','4','5','8','10','13','15','17','18','19']
keep_ints = ['2','6','7','9','11','12','14','16']
keep_dict = {}


#loop keeping ints
for intx in keep_ints:
    #print(f"delete {intx} labels and their images")
    print(f"put {intx} labels and their images into test_new.csv")
    label_quantity = 0
    label_keep_list = []

    #loop label .txts
    for file in os.listdir(label_directory):
        txt_content = open(f'{label_directory}/{file}').read().split()

        #check if keep ints in the label.txt
        if intx in txt_content:
            label_quantity = label_quantity+1
            
            image_name = file[:-4] + ".jpg"
            row_string = image_name + "," + file
            label_keep_list.append([row_string])
            #print(row_string,txt_content)

            #check if delete ints in the label.txt
            for del_intx in delete_ints:
                while del_intx in txt_content:
                    delete_index = txt_content.index(del_intx)
                    txt_content = txt_content[:delete_index] + txt_content[delete_index+5:]
                    #print("DEEEEEEELLLEEEEEEEETTTTTEEEEEEEEE",txt_content)

            #remap labels
            txt_content = [new_label_map.get(item, item) for item in txt_content]

            
            #save label file
            with open(f'/mnt/c/Projects/animal_detection/PASCAL_VOC/labels4/{file}', 'w') as f:
                index = 0

                while index in list(range(0,len(txt_content))):
            
                    content_line = txt_content[index:index+5]
                    #print("write_line:",content_line)
                    f.write(str(content_line).replace('[', '').replace(']', '')\
                                             .replace("'", "").replace(",", "") + '\n')
                    index += 5
            
    keep_dict[intx]={label_quantity:tuple(label_keep_list)}

put 2 labels and their images into test_new.csv
put 6 labels and their images into test_new.csv
put 7 labels and their images into test_new.csv
put 9 labels and their images into test_new.csv
put 11 labels and their images into test_new.csv
put 12 labels and their images into test_new.csv
put 14 labels and their images into test_new.csv
put 16 labels and their images into test_new.csv


In [80]:
def train_test_split(dicx,train_perc):
    print("manual train_test_split")
    for keyx in dicx.keys():

        for label_num in dicx[keyx].keys():

            train_num = int(label_num * train_perc)
            print("**************************************************************")
            print("label:",keyx,"total_input_num",label_num,"split_perc",train_perc)
            all_input = list(dicx[keyx][label_num])
    
            train_input = all_input[:train_num]
            test_input = all_input[train_num:]
    
            print("number of train image inputs",len(train_input),"number of test image inputs",len(test_input))
            try:
                train_input_complete = train_input_complete + train_input
                test_input_complete = test_input_complete + test_input

            except:
                train_input_complete = train_input
                test_input_complete = test_input
        
    return train_input_complete,test_input_complete

In [151]:
train_input_complete,test_input_complete = train_test_split(dicx = keep_dict,train_perc = 0.8)

manual train_test_split
**************************************************************
label: 2 total_input_num 1377 split_perc 0.8
number of train image inputs 1101 number of test image inputs 276
**************************************************************
label: 6 total_input_num 2595 split_perc 0.8
number of train image inputs 2076 number of test image inputs 519
**************************************************************
label: 7 total_input_num 1739 split_perc 0.8
number of train image inputs 1391 number of test image inputs 348
**************************************************************
label: 9 total_input_num 571 split_perc 0.8
number of train image inputs 456 number of test image inputs 115
**************************************************************
label: 11 total_input_num 2125 split_perc 0.8
number of train image inputs 1700 number of test image inputs 425
**************************************************************
label: 12 total_input_num 1043 split_perc 0.

In [139]:
# test_input_complete_noduplicate = [i for n, i in enumerate(test_input_complete) if i not in test_input_complete[:n]]
# train_input_complete_noduplicate = [i for n, i in enumerate(train_input_complete) if i not in train_input_complete[:n]]

# Save train-test inputs w duplications

In [157]:
# opening the csv file in 'w+' mode
file = open('train_new.csv', 'w+', newline ='')
 
# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerows(train_input_complete)

In [158]:
# opening the csv file in 'w+' mode
file = open('test_new.csv', 'w+', newline ='')
 
# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerows(test_input_complete)

## Delete duplication from test.csv

In [202]:
import pandas as pd
test_df = pd.read_csv("test_new.csv", names = ['image','label'])
train_df = pd.read_csv("train_new.csv", names = ['image','label'])

test_df_nod = test_df.drop_duplicates()
train_df_nod = train_df.drop_duplicates()

dupl_elements_df = pd.merge(test_df_nod, train_df_nod, how='inner', on='image')

In [203]:
#perform outer join
outer = test_df_nod.merge(train_df_nod, how='outer', indicator=True)

#perform anti-join
test_df_nod2 = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

In [204]:
len(test_df),len(test_df_nod),len(test_df_nod2)

(3617, 3179, 3078)

In [217]:
len(train_df),len(train_df_nod)

(14453, 12496)

In [218]:
test_df_nod2.to_csv("test_new_clean.csv", encoding='utf-8', index=False, header = False)
train_df_nod.to_csv("train_new_clean.csv", encoding='utf-8', index=False, header = False)

# delete unused images

In [1]:
pwd

'/mnt/c/Projects/animal_detection'

In [5]:
import pandas as pd

test_df = pd.read_csv("test2.csv", names = ['image','label'])
train_df = pd.read_csv("train2.csv", names = ['image','label'])
all_df = pd.concat([test_df,train_df])
all_df.head()

,image,label
0,2009_004898.jpg,2009_004898.txt
1,2009_004946.jpg,2009_004946.txt
2,2009_004959.jpg,2009_004959.txt
3,2009_004996.jpg,2009_004996.txt
4,2009_005040.jpg,2009_005040.txt


In [30]:
used_images_list = all_df['image'].tolist()
len(used_images_list)

18070

In [28]:
import os

image_directory = '/mnt/c/Projects/animal_detection/PASCAL_VOC/images2'

for file in os.listdir(image_directory):
        if file not in used_images_list:
            os.system("rm " f"{image_directory}/{file}")